<a href="https://colab.research.google.com/github/WKLeee/naver_news_crawler/blob/master/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import os
import requests
import csv
import time
from bs4 import BeautifulSoup

In [85]:
# Search Options

## Keyword
keyword = '삼성전자'

## Period
start_date = '2020.09.28'
end_date = '2020.09.28'

## Sort Type (0 = Relevance, 1 = Latest Date, 2 = Oldest Date)
sort_type = '0'

In [97]:
def naver_url_maker(keyword, start_date, end_date):
  naver_url_query = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='
  naver_url_sort = '&sm=tab_opt&sort='
  naver_url_tail = '&photo=0&field=0&reporter_article=&pd=3&ds='
  url = naver_url_query + keyword + naver_url_sort + sort_type + naver_url_tail + start_date + '&de=' + end_date
  return url

# Get Starting url
current_url = naver_url_maker(keyword, start_date, end_date)
print(current_url)

https://search.naver.com/search.naver?where=news&sm=tab_jum&query=삼성전자&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2020.09.28&de=2020.09.28


In [98]:
# Start Crawling
start = time.time()
file_num = 0
pnum = 10 # The number of news pages to  save in one csv file
anum = 0 # Index of Article
csvfile = open('csvfile_'+str(file_num).zfill(3)+'.csv', 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(csvfile)

page_num = 0

update_url = current_url
while True:
  r = requests.get(update_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  link_element = soup.select('.txt_inline > a[href]')

  for link in link_element:
    naver_news_url = link['href'] # Naver News Url
    rn = requests.get(naver_news_url)
    soupn = BeautifulSoup(rn.text, 'html.parser')
    head_tag = soupn.select('.article_info > h3')
    if len(head_tag)>0:
      article_header = head_tag[0].text # Article Title
    body_tag = soupn.select('#articleBodyContents')
    if len(body_tag)>0:
      article_body = body_tag[0].text # Article Body
    time_tag = soupn.select('.t11')
    if len(time_tag)>0: # Article Time
      article_time = time_tag[0].text
    
    wr.writerow([page_num, anum, article_time, article_header, article_body, naver_news_url])
    anum += 1
    
  # If there is no next page, finish.
  pages = soup.select('div.paging a.next')
  if len(pages) == 0:
    break
  else:
    href = pages[len(pages)-1]['href']
    update_url = 'https://search.naver.com/search.naver'+href

  page_num += 1
  if page_num % pnum == 0: # One CSV file has the number of data
    csvfile.close()
    file_num += 1
    csvfile = open('csvfile_'+str(file_num).zfill(3)+'.csv', 'w', encoding='utf-8-sig', newline='')
    wr = csv.writer(csvfile)
    print("Crawling...",anum,"articles...")
    
csvfile.close()
print(time.time()-start)

Crawling... 44 articles...
Crawling... 86 articles...
Crawling... 124 articles...
Crawling... 187 articles...
Crawling... 237 articles...
Crawling... 279 articles...
Crawling... 298 articles...
132.55396270751953
